<h1>
<center>
Data Science Part 1: Wrangle The Data
</center>
</h1>
<div class=h1_cell>
By Fahad Alarefi. The purpose of this three-part project is to use machine learning into making use of a publicaly available table called IMDB 5000 titles.
The main idea is to come up with a function which can predictate, with high accuracy, whether a provided movie data will have 8 stars or more by the user community. The technique of decision trees will be used.
The dataset is accessed from here:
https://data.world/data-society/imdb-5000-movie-dataset

</div>
<br />
<div class=h1_cell>
In this part, we will wrangle the data.
Wrangling the data means to clean the dataset and make it ready for machine learning!
We will be using Pandas library to clean the data and apply the decision trees.
</div>

<h1>
Read the dataset from a public url
</h1>
<div class=h1_cell>
First step,
get the dataset from the web so that we can work on it.
</div>


In [ ]:
import pandas as pd
import os

url = 'https://github.com/createdbyfahad/datascience/files/2593679/movie_metadata.txt'
movie_table = pd.read_csv(url)

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.0f}'.format

<div class=h1_cell>
Here we can print the top 5 rows from the table:
</div>

In [ ]:
movie_table.head()

Check the status of the data (find any empties).


In [ ]:
movie_table.isnull().sum()

Looks like many parts of the data are missing,
however, first we will delete some rows and columns then revaluate empties.
<br />
Since we are interested in modern movies, then first check how many non-color movies are there

In [ ]:
len(movie_table.loc[movie_table['color'] != 'Color'])

 delete all movies that are not in color

In [ ]:
movie_table = movie_table.loc[movie_table['color'] == 'Color']

to make the dataset ligher, we will eliminate all columns that we are not going to use
<br /> 
things like the name of director or the film won't get used in our implementation.

In [ ]:
drop_columns = [
     'color',
     'director_name',
     'num_critic_for_reviews',
     'director_facebook_likes',
     'actor_3_facebook_likes',
     'actor_2_name',
     'actor_1_facebook_likes',
     'actor_1_name',
     'movie_title',
     'num_voted_users',
     'cast_total_facebook_likes',
     'actor_3_name',
     'facenumber_in_poster',
     'plot_keywords',
     'movie_imdb_link',
     'num_user_for_reviews',
     'language',
     'actor_2_facebook_likes',
     'aspect_ratio',
     'movie_facebook_likes'
]
movie_table = movie_table.drop(drop_columns, axis=1)
movie_table.head()

now that we have all important information about the each movie, 
<br />
we will start to prepare it for machine learning
<br />
first, check if how many movies have not content_rating

In [ ]:
len(movie_table.loc[movie_table['content_rating'].isnull()])

since we cannot 'guess' what the movie content type, <br />
then delete all non-rated movies
<br /><br />
also, delete all movies that have not date /OR duration

In [ ]:
movie_table = movie_table.loc[movie_table['content_rating'].notnull()]
movie_table = movie_table.loc[movie_table['title_year'].notnull()]
movie_table = movie_table.loc[movie_table['duration'].notnull()]
len(movie_table)

we are only interested in movies, so drop all tv series<br />
if it has "tv" in it's rating, then drop it

In [ ]:
movie_table = movie_table[~movie_table['content_rating'].str.contains("TV")]
#movie_table = movie_table[movie_table.apply(lambda row: 'TV' not in row['content_rating'], axis=1)]

we are left with good amount of movie titles
<br />
in order to process the genres, we need to convert them into 1s and 0s
<br >
so we will create a column for each possible genre and fill it with 0 or 1
<br />
It's called one-hot encoding

In [ ]:
#movie_table['genres'] = movie_table.genres.str.split('\s*|\s*', expand=True).stack().str.get_dummies().sum(level=0)
movie_table_gd = movie_table['genres'].str.get_dummies(sep='|')
movie_table = movie_table.join(movie_table_gd)
#movie_table += genres
movie_table.head()

drop movies with nan country, <br />
and then one hot encode the coutnry

In [ ]:
movie_table = movie_table.loc[movie_table['country'].notnull()]
one_hot_c = pd.get_dummies(movie_table['country'],prefix='c',dummy_na=False)
movie_table = movie_table.join(one_hot_c)
movie_table.head()

similarly, one hot encode the content_rating <br />
also, I have noticed that some movies use the old rating system (M, GP, G, and X).
<br /> so I will change it to the newer systm

In [ ]:
one_hot_r = pd.get_dummies(movie_table['content_rating'],prefix='rating',dummy_na=False)
movie_table = movie_table.join(one_hot_r)
#movie_table['rating_PG'] = movie_table.apply(lambda row: row['rating_G'])
movie_table['rating_PG'] = (movie_table['rating_G'] | movie_table['rating_PG'])
movie_table['rating_PG-13'] = (movie_table['rating_GP'] | movie_table['rating_M'] | movie_table['rating_PG-13'])
movie_table['rating_NC-17'] = (movie_table['rating_X'] | movie_table['rating_NC-17'])
movie_table['rating_Unrated'] = (movie_table['rating_Not Rated'] | movie_table['rating_Approved'] | movie_table['rating_Unrated'] | movie_table['rating_Passed'])
# then delete unnessasery columns
movie_table = movie_table.drop(['rating_G', 'rating_M', 'rating_GP', 'rating_X', 'rating_Not Rated', 'rating_Approved', 'rating_Passed'], axis=1)
movie_table.head()

Now we are done with content rating, <br />
next, we will classify the numbers data, like 
new_title = if title >= 2000
length_avg = if duration > average movie duration (130 minutes)
good_score = if imdb_score >= 8

In [ ]:
movie_table['is_new'] = movie_table.apply(lambda row: 1 if row.title_year >= 2000.0 else 0, axis=1)
movie_table['is_long'] = movie_table.apply(lambda row: 1 if row.duration > 130.0 else 0, axis=1)
movie_table['is_good'] = movie_table.apply(lambda row: 1 if row.imdb_score >= 8.0 else 0, axis=1)

movie_table.head()

Here are some important functions that will help in applying the kmeans values
(source: Stephen Ficaks, University of Oregon)

In [ ]:
def row_to_vect(row, features):
    vect = []
    for feature in features:
        vect.append(float(row[feature]))
    return tuple(vect)

def initialize_centroids(sample_table, features):
    k = len(sample_table)
    centroids = {}
    for i in range(k):
        row = sample_table.iloc[i]
        vector = row_to_vect(row, features)
        centroids[i] = {'centroid': vector, 'cluster': []}
        
    return centroids

def euclidean_distance(vect1, vect2):
    sum = 0
    for i in range(len(vect1)):
        sum += (vect1[i] - vect2[i])**2
    return sum**.5  # I claim that this square root is not needed in K-means - see why?

def closest_centroid(centroids, row, k):
    min_distance = euclidean_distance(centroids[0]['centroid'], row)
    min_centroid = 0
    for i in range(1,k):
        distance = euclidean_distance(centroids[i]['centroid'], row)
        if distance < min_distance:
            min_distance = distance
            min_centroid = i
    return (min_centroid, min_distance)

def phase_1(centroids, table, features, k):
    for i in range(k):
        centroids[i]['cluster'] = []  # starting new phase 1 so empty out values from prior iteration
    
    #Go through every row in Titanic table (or Loan table) and place in closest centroid cluster.
    for i in range(len(table)):
        row = table.iloc[i]
        vrow = row_to_vect(row, features)
        (index, dist) = closest_centroid(centroids, vrow, k)
        centroids[index]['cluster'].append(vrow)
    
    return centroids

#cluster is a list of points, i.e., a list of lists.
def compute_mean(cluster):
    if len(cluster) == 0:
        return []
    the_sum = cluster[0]  # use 0th point as starter
    
    #I am using zip to pair up all points then do addition
    for i in range(1,len(cluster)):
        the_sum = [pair[0]+pair[1] for pair in zip(the_sum, cluster[i])]
    n = len(cluster)*1.0
    the_mean_point = [x/n for x in the_sum]
    return the_mean_point


def phase_2(centroids, k, threshold):
    old_centroids = []
    
    stop = True
    #Compute k new centroids and check for stopping condition
    for i in range(k):
        current_centroid = centroids[i]['centroid']
        new_centroid = compute_mean(centroids[i]['cluster'])
        centroids[i]['centroid'] = new_centroid
        if euclidean_distance(current_centroid, new_centroid) > threshold:
            stop = False  # all it takes is one

    return (stop, centroids)


def k_means(table, features, k, hypers):
    n = 100 if 'n' not in hypers else hypers['n']
    threshold = 0.0 if 'threshold' not in hypers else hypers['threshold']
    
    centroid_table = table.sample(n=k, replace=False, random_state=100)  # only random choice I am making
    centroids = initialize_centroids(centroid_table, features)
    
    j = 0
    stop = False
    while( j < n and not stop):
        print('starting '+str(j+1))
        centroids = phase_1(centroids, table, features, k)
        (stop, centroids) = phase_2(centroids, k, threshold)
        j += 1
    print('done')
    return centroids


In [ ]:
len(movie_table.loc[movie_table['gross'].isnull()])

gross is not so important field, but it will help us in figuring out the missing budget values
so as you see above, gross has a lot empty values
<br /> so using kmeans, we will fill it with values


In [ ]:
len(movie_table.loc[movie_table['duration'].isnull()])

In [ ]:
features_used = [u'duration',
        u'title_year', u'imdb_score', u'Action', u'Adventure',
       u'Animation', u'Biography', u'Comedy', u'Crime', u'Documentary',
       u'Drama', u'Family', u'Fantasy', u'Film-Noir', u'History', u'Horror',
       u'Music', u'Musical', u'Mystery', u'News', u'Romance', u'Sci-Fi',
       u'Sport', u'Thriller', u'War', u'Western', u'c_Afghanistan',
       u'c_Argentina', u'c_Aruba', u'c_Australia', u'c_Bahamas', u'c_Belgium',
       u'c_Brazil', u'c_Bulgaria', u'c_Cameroon', u'c_Canada', u'c_Chile',
       u'c_China', u'c_Colombia', u'c_Czech Republic', u'c_Denmark',
       u'c_Dominican Republic', u'c_Egypt', u'c_Finland', u'c_France',
       u'c_Georgia', u'c_Germany', u'c_Greece', u'c_Hong Kong', u'c_Hungary',
       u'c_Iceland', u'c_India', u'c_Indonesia', u'c_Iran', u'c_Ireland',
       u'c_Israel', u'c_Italy', u'c_Japan', u'c_Kyrgyzstan', u'c_Mexico',
       u'c_Netherlands', u'c_New Line', u'c_New Zealand', u'c_Norway',
       u'c_Official site', u'c_Panama', u'c_Peru', u'c_Philippines',
       u'c_Poland', u'c_Romania', u'c_Russia', u'c_Slovakia',
       u'c_South Africa', u'c_South Korea', u'c_Spain', u'c_Sweden',
       u'c_Taiwan', u'c_Thailand', u'c_UK', u'c_USA', u'c_West Germany',
       u'rating_NC-17', u'rating_PG', u'rating_PG-13', u'rating_R',
       u'rating_Unrated']


In [ ]:
k = 5
final_centroids = k_means(movie_table, features_used, k, {'n':100, 'threshold':0.0})

now we need to create a table that has all rows with nonempty 'gross' value
we will use that table to get a guess the empty value in our table

In [ ]:
focus_col = 'gross'

nonempty_gross_table = movie_table.loc[movie_table[focus_col].notnull()]
print(len(nonempty_gross_table))

In [ ]:
def compute_centroid_labels(centroids, focus_table, focus_column, features, k):
    for i in range(len(focus_table)):
        row = focus_table.iloc[i]
        vrow = row_to_vect(row, features)
        (minc, mind) = closest_centroid(centroids, vrow, k)
        if focus_column not in centroids[minc]:
            centroids[minc][focus_column] = [row[focus_column]*1.0]
        else:
            centroids[minc][focus_column].append(row[focus_column]*1.0)
    for ind in range(k): 
        if len(centroids[ind][focus_column]) == 0:
            centroids[ind]['mean_label'] = 0.0
        else:
            the_sum = centroids[ind][focus_column][0]
            for i in range(1, len(centroids[ind][focus_column])):
                the_sum += centroids[ind][focus_column][i]
            centroids[ind]['mean_label'] = the_sum/(len(centroids[ind][focus_column]) * 1.0)
            
    return centroids
    
centroids = compute_centroid_labels(final_centroids, nonempty_gross_table, focus_col, features_used, k)

now we will fill the empty 'gross' values in the table.


In [ ]:
def kmeans_fill(centroids, full_table, features, focus_column, k):
   
    def get_closest_clmn(row):
        vrow = row_to_vect(row, features)
        (minc, mind) = closest_centroid(centroids, vrow, k)
        
        return centroids[minc]['mean_label']
    
    new_table = pd.DataFrame(full_table)
    new_table['kmeans_'+focus_column] = new_table.apply(lambda row: get_closest_clmn(row) if pd.isnull(row[focus_column]) else row[focus_column], axis=1)
    
    return new_table

movie_table = kmeans_fill(centroids, movie_table, features_used, focus_col, k)
movie_table.loc[movie_table[focus_col].isnull()].head(20)

As you see above, all empty 'gross' values have kmeans value in the same row
<br /> <br />
now we will do the same thing for the budget column and get a kmeans estimate
<br /> 
but since we have a 'gross' value for every movie, then we will use that info <br />
to gain better estimate for the budget.

In [ ]:
features_used.append('kmeans_gross')

we will redo what we last did, but now try to get the budget value

In [ ]:
focus_col = 'budget'

nonempty_budget_table = movie_table.loc[movie_table[focus_col].notnull()]
print(len(nonempty_budget_table))

In [ ]:
second_final_centroids = k_means(movie_table, features_used, k, {'n':100, 'threshold':0.0})
second_centroids = compute_centroid_labels(second_final_centroids, nonempty_budget_table, focus_col, features_used, k)

In [ ]:
movie_table = kmeans_fill(second_centroids, movie_table, features_used, focus_col, k)
movie_table.loc[movie_table[focus_col].isnull()].head(20)

Now, we are going to use this data to further examination. <br />
In the the next two parts we will try two models of prediciton. (Random Forests and Decision Trees) <br />
In order to use reuse this data, we need to store in a file.


In [ ]:
import os

movie_table.to_csv("part1_movie_table.csv", index=False)